In [5]:
# v tomto kode nacitame udaje o Biological Process, Cellular Component, Molecular Function a konvertujeme ich 
# do RDF grafu "https://pcf_ontology.org"

# importujeme potrebne kniznice
from rdflib import Graph, Dataset, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS, OWL, SKOS

# vytvorime prazdny RDF graf a nacitame obsah suboru go.owl do neho
g = Graph()
g.parse("/Users/simonsnopko/Desktop/Škola/bakalarska praca/go.owl")

# definujeme namespaces pre oboinowl a bioregistry
oboInOwl = Namespace("http://www.geneontology.org/formats/oboInOwl#")
BIOREG = Namespace("https://bioregistry.io/")

# vytvorime RDF dataset a priradime mu meno
ds = Dataset()
graph_name = URIRef("http://pcf_ontology.org")
new_g = ds.graph(identifier=graph_name)
new_g.bind("skos", SKOS)
new_g.bind("bioregistry", BIOREG)

# prechadzame cez vsetky subjekty v povodnom grafe, ktore s typu OWL.Class
for c in g.subjects(RDF.type, OWL.Class):
    # ziskame ID triedy z oboInOwl.id pre dany subjekt
    class_id = g.value(c, oboInOwl.id)
    
    # ak existuje class_id
    if class_id:
        # vytvorime URI pre danu triedu v bioregistry
        bioregistry_uri = URIRef(f"{BIOREG}{class_id}")
    
    # ziskame nazov triedy (label) a pridame tripletu do grafu new_g
    label = g.value(c, RDFS.label)
    if label:
        new_g.add((bioregistry_uri, SKOS.prefLabel, Literal(label)))
    
    # ziskame subClassOf a pridame tripletu do grafu new_g
    subClassOf = g.value(c, RDFS.subClassOf)
    if subClassOf:
        subClassOf_id = "go:" + str(subClassOf).split("GO_")[-1]
        new_g.add((bioregistry_uri, SKOS.narrower, Literal(subClassOf_id)))
        
        # pridame CURIE (notation) do grafu new_g
        new_g.add((bioregistry_uri, SKOS.notation, Literal("go:" + class_id.split("GO:")[-1])))
        
# ulozime vytvoreny RDF dataset do suboru vo formate nquads
ds.serialize(destination="proces_component_function_created_ontology.nq", format="nquads")


<Graph identifier=N4c442dfdc57e40e383fb404f87c856f3 (<class 'rdflib.graph.Dataset'>)>